In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from researchlib.single_import import *

In [2]:
ctf = [transforms.RandomCrop(32, padding=4), transforms.RandomHorizontalFlip()]

# Load dataset
train_loader = FromPublic('cifar10', 'train', batch_size=512, num_workers=8, custom_tf=ctf)
test_loader = FromPublic('cifar10', 'test', batch_size=512, num_workers=8)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Model
model = SENet18(input_dim=3)

# Learning
runner = Runner(model, train_loader, test_loader, optimizer='adam', loss_fn='focal')

In [4]:
for _ in range(3): runner.fit_onecycle(1e-3, mixup_alpha=0.4)

Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
Traceback (most recent call la

KeyboardInterrupt: 

In [5]:
runner.fit_cycle(5, 1e-3, mixup_alpha=0.4)

train_loss: 13.1835,  train_acc: 0.3212,  val_loss: 9.3644,  val_acc: 0.4774


train_loss: 10.9626,  train_acc: 0.4359,  val_loss: 7.5439,  val_acc: 0.5916


train_loss: 9.3133,  train_acc: 0.5376,  val_loss: 6.0860,  val_acc: 0.6703


train_loss: 9.2269,  train_acc: 0.5468,  val_loss: 5.9957,  val_acc: 0.6837


train_loss: 7.9078,  train_acc: 0.6209,  val_loss: 5.0114,  val_acc: 0.7330


train_loss: 7.4283,  train_acc: 0.6536,  val_loss: 4.6078,  val_acc: 0.7688


Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/conda/lib/python3.6/multiprocessing/connection.py", 

KeyboardInterrupt: 

In [ ]:
runner.validate(metrics=[ConfusionMatrix(plot=True, classes=10)])

In [ ]:
runner.history(plot=True)